In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import re
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import datetime

from sklearn.metrics import mean_squared_error
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import svm
from xgboost import XGBClassifier
from sklearn.utils import shuffle

from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM, Dropout, BatchNormalization
 
from sklearn.metrics import classification_report, confusion_matrix


import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from nltk.corpus import stopwords
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

stop_words = set(stopwords.words('english'))

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))

Using TensorFlow backend.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\j\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\j\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\j\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
File = open('stocks_name.txt','r')

Name = []
Codes = []
Type = []
Add = []

for line in File:
    line = re.sub('\n', '', line)
    line = line.split('\t')
    Name.append(re.sub(' ', ' ',line[0].lower())+' ')
    Type.append(line[2])
    Add.append(line[1].lower()+ ' ')
    if line[1][2:4] == '.A':
        Codes.append('BT-A.L')
    elif line[1][2] == '.':
        Codes.append(line[1] + 'L')
    else:
        Codes.append(line[1] + '.L')


In [15]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()


def window(stock, brokers, size, delay):
    
    windows = []
    binary_pred = []
    pred = []
        
    
    for i in range(len(stock) - delay - size):
        
        scaler.fit(stock[i: i + size])
        
        win = []
        for j in range(size):
            # comment top line to only include brokers opinions
            #win.append(np.concatenate(scaler.transform(stock[i+j:i+j+1])))
            win.append(brokers[i+j])
            
        windows.append(np.concatenate(win))
        
        pred.append(scaler.transform(stock[i+size:i + size + delay])[-1][-3] - scaler.transform(stock[i:i + size])[-1][-3]) # The actual value scaled appropriatly
        
        #Binary pred catagory
        if scaler.transform(stock[i+size:i + size + delay])[-1][-3] < scaler.transform(stock[i:i + size])[-1][-3]:
            binary_pred.append(1)
        else:
            binary_pred.append(0)
    
        
    return windows, binary_pred, pred


In [ ]:

Svm = svm.SVC(C=1)
mod = XGBClassifier()

SVM = []
LOG = []
XGB = []
LSTm = []

fSVM = []
fLOG = []
fXGB = []
fLSTm = []

SPLIT = []

#fix codes
#fix missing
for code in Codes[1:]:

    DF = pd.read_csv('all_data/' + code + '.csv')

    days = 10

    DF['BERT combined'] = DF['Bert'][::1].rolling(days, min_periods=0).sum()[::-1]

    DF = DF.drop(['Bert','neg', 'neu','pos'], axis=1)
    
    DF = DF.dropna()
    
    Brokers = DF[['Buy1', 'Outperform1', 'Hold1', 'Underperform1', 'Sell1', 'BERT combined']].values

    Stock = DF[['Open', 'High', 'Low', 'Close', 'Adj Close','Volume']].values
    
    
    windows, binary_pred, pred = window(Stock, Brokers, 10, 10)
    
    
    X_train = windows[:1000]
    X_test = windows[1000:]
    y_train = binary_pred[:1000]
    y_test = binary_pred[1000:]
    
    if len(y_test) > 0:
    
        split = 0
        for i in y_test:
            if i == 0:
                split+=1
        SPLIT.append(split/len(y_test))

        print('Split : ', split/len(y_test))

        Svm.fit(X_train, y_train)
        y_pred = Svm.predict(X_test)
        SVm = accuracy_score(y_test, y_pred) 
        fSVm = f1_score(y_test, y_pred, average='macro')
        print('SVM : ', SVm, fSVm)

        clf = LogisticRegression(random_state=0, max_iter=1000).fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        log = accuracy_score(y_test, y_pred) 
        flog = f1_score(y_test, y_pred, average='macro')
        print('log reg : ', log, flog)

        mod.fit(np.array(X_train), np.array(y_train))
        y_pred = mod.predict(np.array(X_test))
        xgb = accuracy_score(y_test, y_pred) 
        fxgb = f1_score(y_test, y_pred, average='macro')
        print('XGBoost : ', xgb, fxgb)

        # reshape input to be 3D [samples, timesteps, features]
        train_X = np.array(X_train).reshape((np.array(X_train).shape[0], 1,np.array(X_train).shape[1]))
        test_X = np.array(X_test).reshape((np.array(X_test).shape[0], 1, np.array(X_test).shape[1]))


        Model = Sequential()
        Model.add(LSTM(180, return_sequences=True, input_shape=(train_X.shape[1], train_X.shape[2])))
        Model.add(LSTM(units=90, return_sequences=True))
        #model.add(BatchNormalization())
        Model.add(LSTM(units=30))
        Model.add(Dense(units=1, activation='relu'))

        Model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

        Model.fit(train_X,np.array(y_train), epochs=10, batch_size=32,shuffle = False,validation_split=0.2, verbose=0)

        y_pred = Model.predict(np.array(test_X), verbose=0)

        lstm = accuracy_score(y_test, y_pred.round()) 
        flstm = f1_score(y_test, y_pred.round(), average='macro')
        
        print('LSTM : ', lstm, flstm)

        SVM.append(SVm)
        LOG.append(log)
        XGB.append(xgb)
        LSTm.append(lstm)
        
        fSVM.append(fSVm)
        fLOG.append(flog)
        fXGB.append(fxbg)
        fLSTm.append(flstm)

In [ ]:
print(len(SVM))
code = Codes[1:54] + Codes[55:100]
results = pd.DataFrame({'SVM' : SVM, 'fSVM' : fSVM, 'Logistic reg' : LOG, 'flog reg' : fLOG, 'XBGoost' : XGB, 'fXGB' : fXGB,
                        'LSTM': LSTm, 'flstm' : fLSTm}, index = code)
results.to_csv('RESULTS/S+T.csv')

print('Mean')
print(results.mean())
print('Variance')
print(results.var())

results.head()

In [20]:
delay = 10
days = 10    

X_train = []
X_test = []
y_train = []
y_test = []
binary_train = []
binary_test = []
ALL = []


for code in Codes[1:]:

    DF = pd.read_csv('all_data/' + code + '.csv')

    days = 10

    DF['BERT combined'] = DF['Bert'][::1].rolling(days, min_periods=0).sum()[::-1]

    DF = DF.drop(['Bert','neg', 'neu','pos'], axis=1)
    
    DF = DF.dropna()
    
    
    Brokers = DF[['Buy1','Outperform1', 'Hold1', 'Underperform1', 'Sell1', 'BERT combined']].values

    Stock = DF[['Open', 'High', 'Low', 'Close', 'Adj Close','Volume']].values
    
    windows, binary_pred, pred = window(Stock, Brokers, 10,10)
    
    print(len(windows),len(pred), len(binary_pred))
    
    y_train = y_train + pred[:1000]
    y_test = y_test + pred[1000:]
    
    binary_train = binary_train + binary_pred[:1000]
    binary_test = binary_test + binary_pred[1000:]
        
    X_train = X_train + windows[:1000]
    X_test = X_test + windows[1000:]
    
    ALL = ALL + binary_pred

1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1252 1252 1252
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1470 1470 1470
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
94 94 94
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1

In [21]:

clf = LogisticRegression(random_state=0, max_iter=1000).fit(X_train, binary_train)
y_pred = clf.predict(X_test)
log = accuracy_score(binary_test, y_pred) 
flog = f1_score(binary_test, y_pred, average='macro')
print('log reg : ', log, flog)

mod = XGBClassifier()
mod.fit(np.array(X_train), np.array(binary_train))
y_pred = mod.predict(np.array(X_test))
xgb = accuracy_score(binary_test, y_pred) 
fxgb = f1_score(binary_test, y_pred, average='macro')
print('XGBoost : ', xgb, fxgb)

# reshape input to be 3D [samples, timesteps, features]
train_X = np.array(X_train).reshape((np.array(X_train).shape[0], 1,np.array(X_train).shape[1]))
test_X = np.array(X_test).reshape((np.array(X_test).shape[0], 1, np.array(X_test).shape[1]))

Model = Sequential()
Model.add(LSTM(180, return_sequences=True, input_shape=(train_X.shape[1], train_X.shape[2])))
Model.add(LSTM(units=90, return_sequences=True))
#Model.add(BatchNormalization())
Model.add(LSTM(units=30))
Model.add(Dense(units=1, activation='relu'))

Model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Model.fit(train_X,np.array(binary_train), epochs=1, batch_size=32, shuffle = False, validation_split=0.2, verbose=0)

y_pred = Model.predict(np.array(test_X), verbose=0)

lstm = accuracy_score(binary_test, y_pred.round()) 
flstm = f1_score(binary_test, y_pred.round(), average='macro')        
print('LSTM : ', lstm, flstm)


log reg :  0.5104265139298226 0.33840914425369206
XGBoost :  0.5144488313036386 0.43840480196720266
LSTM :  0.5111494188956236 0.3408151557425376


S T B
log reg :  0.5112606350442084 0.34941544900102334
XGBoost :  0.5122430443567073 0.4616196911442263
LSTM :  0.5103709058555302 0.33791097529546044

S T
log reg :  0.5122986524309997 0.3832466908666927
XGBoost :  0.5074236779180337 0.4856619682102835
LSTM :  0.5103709058555302 0.33791097529546044

S B
log reg :  0.510760162375577 0.3464103395497862
XGBoost :  0.5078314704628445 0.44996157555432986
LSTM :  0.5103709058555302 0.33791097529546044

S
log reg :  0.5109269865984541 0.3774555105464708
XGBoost :  0.504587666129122 0.48119366636018585
LSTM :  0.510352369830766 0.33800445440691235

B
log reg :  0.510352369830766 0.33797059209975344
XGBoost :  0.513633246214017 0.4100036946837216
LSTM :  0.5103709058555302 0.33791097529546044

T
log reg :  0.510630410202228 0.3387015745709705
XGBoost :  0.5160429294333537 0.435298519919782
LSTM :  0.5126508369015181 0.3507383333606752

B T
log reg :  0.5104265139298226 0.33840914425369206
XGBoost :  0.5144488313036386 0.43840480196720266
LSTM :  0.5111494188956236 0.3408151557425376